# CSCI4022 Homework 4; Item Sets

## Due Friday, March 12 at 11:59 pm to Canvas

#### Submit this file as a .ipynb with *all cells compiled and run* to the associated dropbox.

***

Your solutions to computational questions should include any specified Python code and results as well as written commentary on your conclusions.  Remember that you are encouraged to discuss the problems with your classmates, but **you must write all code and solutions on your own**.

**NOTES**: 

- Any relevant data sets should be available on Canvas. To make life easier on the graders if they need to run your code, do not change the relative path names here. Instead, move the files around on your computer.
- If you're not familiar with typesetting math directly into Markdown then by all means, do your work on paper first and then typeset it later.  Here is a [reference guide](https://math.meta.stackexchange.com/questions/5020/mathjax-basic-tutorial-and-quick-reference) linked on Canvas on writing math in Markdown. **All** of your written commentary, justifications and mathematical work should be in Markdown.  I also recommend the [wikibook](https://en.wikibooks.org/wiki/LaTeX) for LaTex.
- Because you can technically evaluate notebook cells is a non-linear order, it's a good idea to do **Kernel $\rightarrow$ Restart & Run All** as a check before submitting your solutions.  That way if we need to run your code you will know that it will work as expected. 
- It is **bad form** to make your reader interpret numerical output from your code.  If a question asks you to compute some value from the data you should show your code output **AND** write a summary of the results in Markdown directly below your code. 
- 45 points of this assignment are in problems.  The remaining 5 are for neatness, style, and overall exposition of both code and text.
- This probably goes without saying, but... For any question that asks you to calculate something, you **must show all work and justify your answers to receive credit**. Sparse or nonexistent work will receive sparse or nonexistent credit. 
- There is *not a prescribed API* for these problems, except the **form of your output for #3**.  You may answer coding questions with whatever syntax or object typing you deem fit.  Your evaluation will primarily live in the clarity of how well you present your final results, so don't skip over any interpretations!  Your code should still be commented and readable to ensure you followed the given course algorithm.

---

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm


***
<a/ id='p1'></a>
[Back to top](#top)
# Problem 1 (Theory: A Priori Properties; 10 pts) 
In using triangular arrays to store item basket data, we defined the function $a[k] :=$ count for the pair ${i, j}$, where $1 \leq i < j \leq n,$ with

$k=(i-1)\left(n-\frac{i}{2}\right) +j -i$


This formula involves dividing an arbitrary integer i by 2. Yet $k$ is an index, so we need to have k be an integer. Prove that k will, in fact, be an integer.


So the first thing to note is i and j are integers so addition or subtraction of either wont affect k being an integer. Now lets focus on just the first half of the formula $f(i)=(i-1)\left(n-\frac{i}{2}\right)$:
Lets go through each possible case for k...
- Case 1: i=1 $f(1) = (1-1)(n-\frac{1}{2}) = 0$
- Case 2: i=2 $f(2) = (2-1)(n-\frac{2}{2}) = n - 1$, this happens to be the exact amount of elements in the first row of our upper triangular matrix
- Case 3: i=3 $f(3) = (3-1)(n-\frac{3}{2}) = 2n - 3 = (n-1)+(n-2)$, this corresponds to now the first two rows of upper triangular matrix 
- Case 4: i=4 $f(4) = (4-1)(n-\frac{4}{2}) = 3n - 6= (n-1)+(n-2)+(n-3)$
- Case 4: i=k $f(k) = (k-1)(n-\frac{k}{2}) = \sum_{k=1}^{n}(n-k)$

As you can now see the developing pattern, no matter the input i as long as it is in the current constraints no value of i would make f(i) not produce an integer. f(i) is designed to record all the elements before the current k, hence k will be an integer. 


***
<a/ id='p2'></a>
[Back to top](#top)
# Problem 2 (Theory: Item Baskets; 10 pts)

Suppose we have 20 distinct items numbered 1 to 20. Each basket is constructed by including the item numbered `k` with probability $1/k$, independent of other items.  As a result, all baskets will include item 1, half will include item 2, and so forth.  What are all of the *itemsets* expected to be frequent at a support threshold of 3%?

Note: You may use simulation if you prefer, but I suspect you may find the pen-and-paper answer is easier.

So if we have 20 baskets are support threshold in terms of baskets is 20 * .03 = .6. \
So if we had n baskets are support threshold in terms of baskets is n * .03 \
I thought simulation would be more fun...

In [2]:
def getBasket():
    itemHereStates= [0,1]
    basket=[0]*20
    #print(len(basket))
    for i in range(1,21):
        #print(1/i)
        choice=np.random.choice([0,i],p=[1-1/i,1/i])

        basket[i-1]=choice
    #print(basket)
    return basket
def oneSim(n_baskets):
    nBasketMatrix=[]
    for i in range(n_baskets):
        nBasketMatrix.append(getBasket())
    return nBasketMatrix



def makeTrip(Triang_Counts, s):
    Candidates=[]
    freqItemPairs = []
    finalPairs=[]
    size= len(Triang_Counts)
    rows = 20
    
    #grabbed all frequent item pairs
    for idx1 in range(rows):
        for idx2 in range(idx1+1, rows):
            k = int(idx1*(rows - (idx1+1)/2) + idx2 - idx1 - 1) #k represent index i,j 
            #print(k)
            #print(s)
            if(Triang_Counts[k] >= s):
                freqItemPairs.append((idx1, idx2, Triang_Counts[k]))
                finalPairs.append((idx1, idx2))
    print("Frequent Itempairs as Arrays of Triples")
    print(freqItemPairs)
    print("Expected Itemsets at threshold of 3%:")
    print(finalPairs)

def finalSim(n):
    matrix1 = oneSim(n)
    #calculate f
    #df = pd.DataFrame(data=matrix1)
    #df1 = np.array(df)
    frequencies=[0]*20
    U = np.zeros((20, 20))
    U_single= np.zeros(20)
    for irow in range(20):
        U_single[irow]=np.sum([set([irow+1]) <= set(matrix1[k]) for k in range(len(matrix1))])
        for icol in range(irow+1, 20):
            item1 = irow+1
            item2 = icol+1
            U[irow,icol] = np.sum([set([item1, item2]) <= set(matrix1[k]) for k in range(len(matrix1))])
    #create triangular array as 1-d array
    nt = int((20-1)*20/2)
    a = [0]*nt
    k = 0
    for i in range(20):
        for j in range(i+1, 20):
            a[k] = U[i,j]
            k += 1
    print("With "+str(n)+" as the number of baskets simulated:")
    print("Our support threshold at 3% would correspond to "+str(int(.03*n)) + " baskets")
    print("singleton itemset count:")
    print(U_single)
    print("All singleton itemsets would be considered frequent")
    print("________________________________________________________")
    print("More interesting would be itemsets as item pairs...")
    print("triangular array of items pairs as 1-d array")
    print(a)

    makeTrip(a,.03*n)
n=10000 #10000
finalSim(n)

With 10000 as the number of baskets simulated:
Our support threshold at 3% would correspond to 300 baskets
singleton itemset count:
[10000.  5095.  3377.  2483.  1960.  1683.  1426.  1279.  1047.   932.
   955.   839.   813.   744.   671.   609.   562.   575.   526.   513.]
All singleton itemsets would be considered frequent
________________________________________________________
More interesting would be itemsets as item pairs...
triangular array of items pairs as 1-d array
[5095.0, 3377.0, 2483.0, 1960.0, 1683.0, 1426.0, 1279.0, 1047.0, 932.0, 955.0, 839.0, 813.0, 744.0, 671.0, 609.0, 562.0, 575.0, 526.0, 513.0, 1737.0, 1228.0, 953.0, 841.0, 736.0, 629.0, 523.0, 474.0, 485.0, 420.0, 426.0, 365.0, 359.0, 300.0, 279.0, 271.0, 277.0, 263.0, 822.0, 673.0, 564.0, 472.0, 468.0, 347.0, 315.0, 314.0, 261.0, 295.0, 258.0, 215.0, 197.0, 195.0, 191.0, 181.0, 178.0, 501.0, 411.0, 352.0, 354.0, 271.0, 239.0, 230.0, 216.0, 216.0, 180.0, 178.0, 164.0, 127.0, 169.0, 121.0, 135.0, 333.0, 277.0, 251.

***
<a/ id='p3'></a>
[Back to top](#top)
# Problem 3 (Practice: Candidate Items; 25 pts)

In the A-Priori algorithm, there is a step in which we create a candidate list of frequent itemsets of size $k+1$ as we prune the frequent itemsets of size $k$.  This this problem we will create two functions to do that formally.

#### Part A:

There are two types of data objects in which we might be holding the frequency counts of itemsets.  If $k=2$, they may be stored in a triangular array.  Create a function `Cand_Trips` that takes a triangular array and returns all valid candidate triples as a list.  Recall that the itemset $\{i,j,k\}$ is only a candidate if all 3 of the itemsets in $\{\{i,j\}, \{i,k\}, \{k,j\}\}$ are frequent.

Some usage notes:

- The first input argument is `Triang_Counts`,  a zero-indexed triangular (numeric) array, by same convention as introduced in class.
- The second input argument is the positive integer support threshold `s`.
- The underlying itemset is 0-indexed, so e.g. `[0,1,3]` is a valid triple.
- The return array `Candidates` should be a list of 3-index lists of the item numbers of the triples.  So a final answer for some input might be:

`Cand_Trips` =
    `[[0,3,4], [1,2,7]]`

- An implementation note: there are two fundamentally different ways to think about implementing this function.  Option 1 involves thinking about the elements of `Tri_Counts` in terms of their locations on the corresponding *triangular matrix*: scan row $i$ for a pair of frequent pairs $\{\{i,j\}, \{i,k\}\}$ and then check if $\{j,k\}$ is in fact frequent.  Option 2 scans all of `Tri_Counts` for frequent item pairs (the "pruning" step) and saves those in some object with their indices, then scans *that* object for candidates.  Both are valid for this problem, but option 2 may generalize to higher $k$ better...

In [3]:
def Cand_Trips(Triang_Counts, s):
    #do the thing
    Candidates=[]
    freqItemPairs = []
    size= len(Triang_Counts)
    rows = int(len(Triang_Counts)/2)
    
    #grabbed all frequent item pairs
    for idx1 in range(rows):
        for idx2 in range(idx1+1, rows):
            k = int(idx1*(rows - (idx1+1)/2) + idx2 - idx1 - 1) #k represent index i,j 
            #print(k)
            freqItemPairs.append((idx1, idx2, Triang_Counts[k]))
    #print(freqItemPairs)
    Candidates = possibleCandidates(freqItemPairs,s) #helper function to find valid canditates
    
    return Candidates
def possibleCandidates(freqItemPairs,s): #return a list [i,j,k]
    L1 = []#pairs that beat threshold
    C1 =[]#final trips list
    for freqItemPair in freqItemPairs:
        if(freqItemPair[2]>= s):
            i = freqItemPair[0]
            j = freqItemPair[1]
            L1.append({i,j})
    
    for i in range(len(L1)):
        pair = L1[i]
        for j in range(len(L1)):
            if(i != j) and (pair.intersection(L1[j])):
                trip = list(pair.union(L1[j]))
                otherPair= pair.union(L1[j])
                #print(otherPair)
                otherPair.remove(list(pair.intersection(L1[j]))[0])
                #print(otherPair)
                #print(list(pair.intersection(L1[j]))[0])
                if(trip not in C1) and (otherPair in L1) :
                    
                    #print(str(pair) + " vs "+str(L1[j])+" : " + str(pair.intersection(L1[j])) )
                    #print(trip)
                    C1.append(trip)
            
    #print(L1)            
    #print(C1)
    return C1

#### Part B:

A quick test case.  Below is  a matrix $M$ and code including its corresponding the triangular array.  

$C=\begin{bmatrix}
\cdot &10&7&3&2\\
\cdot &\cdot&6&4&3\\
\cdot &\cdot&\cdot&3&6\\
\cdot &\cdot&\cdot&\cdot&0\\
\cdot &\cdot&\cdot&\cdot&\cdot\\
\end{bmatrix}$
 
Input the given list into your function to verify that it returns the correct valid triples at $s=1$ and $s=6$.

In [4]:
Triang_Counts=[10,7,3,2,6,4,3,3,2,0]

#Check that...
#Cand_Trips(Triang_Counts, 1) returns all the possible triples except those that contain BOTH items 3 and 4.
#Cand_Trips(Triang_Counts, 6) returns only the triple [[0,1,2]]
Cand_Trips(Triang_Counts, 1)

[[0, 1, 2], [0, 1, 3], [0, 1, 4], [0, 2, 3], [0, 2, 4], [1, 2, 3], [1, 2, 4]]

In [5]:
Cand_Trips(Triang_Counts, 6)

[[0, 1, 2]]

#### Part C:

Suppose instead that our $k=2$ item counts were stored in a list of the form e.g.
`Pairs_Counts` =
    `[[0,1,12], [0,2,0], [0,3,11], ..., [7,8,103]]`
    
Where each element is a triple storing the two item indices and their count, $[i,j,c_{ij}]$. 

Create a function `Cand_Trips_List` that takes in a list of pairs counts and returns all valid candidate triples as a list.  

Some usage notes:

- The first input argument is `Pairs_Counts`,  a zero-indexed list of triples.
- The second input argument is the positive integer support threshold `s`.
- The underlying itemset is 0-indexed, so e.g. `[0,1,3]` is a valid triple.
- The return array `Candidates` should be a list of 3-element lists, as above.

You should **not** convert the input list `Pairs_Counts` into a triangular array as part of your function.  After all, sometimes we use the list format for pairs because it saves memory compared to the triangular array format!  You may be able to borrow heavily from the logic of your first function, though!

In [6]:
def Cand_Trips_List(Pairs_Counts, s):
    #do the thing
    Candidates = possibleCandidates(Pairs_Counts,s) #wrote this function possibleCanditates to use in part b so I just called here
    return Candidates



#### Part D:

Do the test case again.  Below is the list reprentation of the same matrix $M$ from part B.  
 
Input the given list into your function to verify that it returns the correct valid triples at $s=1$ and $s=6$.

In [7]:
Pairs_Counts=[[0,1,10], [0,2,7], [0,3,3], [0,4,2],\
             [1,2,6],[1,3,4], [1,4,3],\
             [2,3,3],[2,4,2],\
             [3,4,0]]
Pairs_Counts

#Check that...
#Cand_Trips(Pairs_Counts, 1) returns all the possible triples except those that contain BOTH items 3 and 4.
#Cand_Trips(Pairs_Counts, 6) returns only the triple [[0,1,2]]

[[0, 1, 10],
 [0, 2, 7],
 [0, 3, 3],
 [0, 4, 2],
 [1, 2, 6],
 [1, 3, 4],
 [1, 4, 3],
 [2, 3, 3],
 [2, 4, 2],
 [3, 4, 0]]

In [8]:
print("Cand_Trips_List(Pairs_Counts, 1): " +str(Cand_Trips_List(Pairs_Counts, 1)))
print("Cand_Trips_List(Pairs_Counts, 6): " +str(Cand_Trips_List(Pairs_Counts, 6)))


Cand_Trips_List(Pairs_Counts, 1): [[0, 1, 2], [0, 1, 3], [0, 1, 4], [0, 2, 3], [0, 2, 4], [1, 2, 3], [1, 2, 4]]
Cand_Trips_List(Pairs_Counts, 6): [[0, 1, 2]]


#### Part E

Describe *in words* how you would generalize your code in part D to work for generating candidate quadruples $[i_1, i_2, i_3, i_4]$ from an input list of triples counts (each element of the form $[i, j, k, c_{ijk}]$).

The way my code works (refering to function "possibleCandidates" in part b as it solves part D) is with two main loops. The first one is to collect a list of each element where $c_{ijk} > s $ in the form of a set {i, j, k}. The only real change I would have to make to this for loop is to make it accept n amount of elements(in this case n=4). Now that we have the elements that our "frequent", stored in list L1. We need to get our candidate quadruples. Using the same definition tweaked for quadruples:we get that the itemset $\{i_1, i_2, i_3, i_4\}$ is only a candidate if all 4 of the itemsets in $\{\{i_1, i_2, i_3\}, \{i_1, i_2, i_4\}, \{i_1, i_3, i_4\}, \{i_2, i_3, i_4\}\}$ are frequent. Before we enter the second loop, we need to store the candidates in a list lets say C1. The second main loop is a nested loop that iterates over each triple pair in L1 and for each pair checks every other pair in L1 to see if they intersect with another. If that is true than using a few set operations we can find the other two pairs and if both are in L1 and not in C1(no duplicates), we append our new found set in list form to C1. 

# Problem 4 (Practice: A-Priori.  Not due this week.)

This problem is **not on this homework**.  It is repeated on the homework due next week on Friday, Mar 19, which also contains your first PageRank/power iteration problem(s).  But it is contained here in brief in case you wish to start on it early, because it involves using A-Priori and includes some data wrangling/munging that you might enjoy extra time on.

Consider the Online Retail data set provided in `onlineretail.csv`.  This includes over 500,000 purchases from an online retailer.

We want to use the baskets (marked by `InvoiceNo`) and the items (marked by `StockCode` and/or `Description`) to perform an item basket analysis.

This data set is small enough to run directly from main memory, so you may do that if you wish.  You may also complete this problem using only the first 100,000 entries of the .csv if you wish for shorter computational time.  Be very explicit which you are using.

#### a)  There are some odd entries in the data set.  Make sure that you're discarding any transactions and items with no `Description`, non-positive `Quantity`, or non-positive `Unit Price`.


#### b) For our first iteration, we will use just `StockCode` for the items.  Use `StockCode` to create a table of frequent single items at 1% support threshold.  For convenience on this part of the problem and part c), you may choose to discard all items with non-integer values in `StockCode`.  Was 1% an appropriate support threshold?  Describe why or why not.

#### c) Use A-priori to find all frequent  pairs of items from your set of frequent items in a).  Use whatever support threshold you feel is most appropriate.

#### d) Use a hash table to hash items from their `Descriptions`.  Include a check to minimize and fix any collisions, as in nb08.

#### e) Use A-priori to find all frequent items and all frequent pairs of items from your hashed data set in part c).

#### f) Did any frequent items appear in part d) that did not in part b)?  If so, list them.